<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/cnn2%20(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow_addons

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_addons as tfa

In [ ]:


# Load MNIST dataset
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize the images to [-1, 1]
#x_train = (x_train.astype("float32") - 127.5) / 127.5
#y_train = y_train.reshape(-1, 1)
x_train = (x_train.astype("float32")) / np.max(x_train)

In [ ]:
new_images = []
new_labels = []

for digit in set(y_train):
    digit_indices = np.where(y_train == digit)[0][:10]
    for index in digit_indices:
        image = x_train[index]
        label = y_train[index]
        new_images.append(image)
        new_labels.append(label)

new_images = np.array(new_images)
new_labels = np.array(new_labels)

# Create an array with indices from 0 to length of your arrays
indices = np.arange(new_images.shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to shuffle your arrays
new_images = new_images[indices]
new_labels = new_labels[indices]

print("New dataset shape:", new_images.shape)

# Visualize the new dataset
plt.imshow(new_images[30], cmap="gray")
plt.title("Label: {}".format(new_labels[30]))
plt.axis("off")
plt.show()

In [ ]:
def random_rotate_images(images, min_angle=-90):
    size = tf.shape(images)[0]
    random_angles = tf.random.uniform(shape=(size,), minval=min_angle, maxval=-min_angle, dtype=tf.float32)
    rotated_images = tfa.image.rotate(images, random_angles / 90)
    return rotated_images

# Select a batch of images
batch_size = 10
batch_images = x_train[:batch_size]
batch_images = np.expand_dims(batch_images, axis=-1)
rotated_images = random_rotate_images(batch_images)

fig, axes = plt.subplots(nrows=2, ncols=batch_size, figsize=(10, 4))
for i in range(batch_size):
    axes[0][i].imshow(batch_images[i], cmap='gray')
    axes[0][i].axis('off')
    axes[1][i].imshow(rotated_images[i], cmap='gray')
    axes[1][i].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
def build_generator():
    noise_shape = (100,)
    noise = layers.Input(shape=noise_shape)
    input_digit = layers.Input(shape=(1,), dtype="int32")
    digit_embedding = layers.Embedding(input_dim=10, output_dim=64, input_length=1)(input_digit)
    digit_embedding = layers.Flatten()(digit_embedding)

    x = layers.Concatenate()([noise, digit_embedding])

    # Transform the concatenated vector into a 7x7x256 tensor
    x = layers.Dense(7 * 7, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7, 7, 1))(x)

    # Upsample to 14x14
    x = layers.Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Conv 14x14
    x = layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Upsample to 28x28
    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Final output layer
    output = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='hard_sigmoid')(x)

    model = tf.keras.models.Model(inputs=[noise, input_digit], outputs=output)
    return model

generator = build_generator()
generator.summary()


In [ ]:
def build_discriminator():
    input_image = layers.Input(shape=(28, 28, 1))
    digit_input = layers.Input(shape=(1,), dtype="int32")
    digit_embedding_1d = layers.Embedding(input_dim=10, output_dim=64, input_length=1)(digit_input)
    digit_embedding_1d = layers.Flatten()(digit_embedding_1d)
    digit_embedding_2d = layers.Embedding(input_dim=10, output_dim=28*28, input_length=1)(digit_input)
    digit_embedding_2d = layers.Reshape((28,28,1))(digit_embedding_2d)

    merged_input = layers.Concatenate()([input_image, digit_embedding_2d])

    # Convolutional layers
    x = layers.Conv2D(128, (2, 2), strides=(2, 2), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(merged_input)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.8)(x)

    x = layers.Conv2D(248, (5, 5), strides=(2, 2), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.8)(x)

    x = layers.Conv2D(248, (5, 5), strides=(2, 2), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.8)(x)

    x = layers.Flatten()(x)
    x = layers.Concatenate()([x,digit_embedding_1d])
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(300, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.8)(x)
    x = layers.Dense(1, activation="sigmoid", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

    model = tf.keras.models.Model(inputs=[input_image, digit_input], outputs=x)
    return model

discriminator = build_discriminator()
discriminator_copy = build_discriminator()
#discriminator.summary()


In [ ]:
# Compile models
generator_optimizer = tf.keras.optimizers.Adam(0.0004)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)
copy_optimizer = tf.keras.optimizers.Adam(0.0004)

#generator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)
#discriminator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)



In [ ]:
def print_img(generator_model):
    # Generate and save sample images
    noise = tf.random.normal([10, 100])
    sampled_labels = tf.constant([[i % 10] for i in range(10)], dtype=tf.int32)
    generated_images = generator_model.predict([noise, sampled_labels])
    fig, axs = plt.subplots(1, 10, figsize=(10, 10))
    for i in range(10):
        axs[i].imshow(generated_images[i], cmap="gray")
        axs[i].axis("off")
    plt.show()

In [ ]:
#@tf.function
def train_step(images, labels):

    batch_size = images.shape[0]
    noise = tf.random.normal([batch_size, 100])
    generated_images = generator([noise, labels], training=True)



    with tf.GradientTape() as disc_tape:

      real_output = discriminator([images, labels], training=True)
      fake_output = discriminator([generated_images, labels], training=True)

      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


    # Get the weights of model1
    weights = discriminator.get_weights()

    # Set the weights of model2 to be the same as model1
    discriminator_copy.set_weights(weights)

    for i in range(5):
      with tf.GradientTape() as disc_copy_tape:
        noise = tf.random.normal([batch_size, 100])
        generated_images = generator([noise, labels], training=True)

        real_output = discriminator_copy([images, labels], training=True)
        fake_output = discriminator_copy([generated_images, labels], training=True)

        disc_loss = discriminator_loss(real_output, fake_output)

      gradients_of_discriminator = disc_copy_tape.gradient(disc_loss, discriminator_copy.trainable_variables)
      copy_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator_copy.trainable_variables))



    with tf.GradientTape() as gen_tape:
      noise = tf.random.normal([batch_size, 100])
      generated_images = generator([noise, labels], training=True)
      fake_output = discriminator_copy([generated_images, labels], training=True)
      gen_loss = generator_loss(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))


    tf.print("disc_loss",disc_loss,'gen_loss',gen_loss)


def train(generator, discriminator
          , epochs, batch_size):
    for epoch in range(epochs):
        for batch in range(len(new_images) // batch_size):
            images = new_images[batch * batch_size: (batch+1) * batch_size]
            labels = new_labels[batch * batch_size: (batch+1) * batch_size]

            train_step(images, labels)

        # Output training progress
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print_img(generator)

# Train the GAN
EPOCHS = 2000000
BATCH_SIZE = 50
num_unrolling_steps = 5  # Set the desired number of unrolling steps
train(generator, discriminator, EPOCHS, BATCH_SIZE)